### Import Libraries

In [1]:
import os
import tensorflow as tf
from pathlib import Path

### Input Variables

In [2]:
DATA_DIR = "/Users/leonardo/Documents/Projects/cryptovision/data/processed"

SEED = 123

IMAGE_RESOLUTION = (224, 224)

BATCH = 64

EPOCHS = 20

train_dir = DATA_DIR + "/train"
valid_dir = DATA_DIR + "/valid"
test_dir = DATA_DIR + "/test"

pretrain_models = {
    "MOBV2": {
        "model": tf.keras.applications.MobileNetV2,
        "prep": tf.keras.applications.mobilenet_v2.preprocess_input,
    },
    "RES50V2": {
        "model": tf.keras.applications.ResNet50V2,
        "prep": tf.keras.applications.resnet_v2.preprocess_input,
    },
    "EFFV2B0": {
        "model": tf.keras.applications.EfficientNetV2B0,
        "prep": tf.keras.applications.efficientnet_v2.preprocess_input,
    },
}

NEURONS_LAYERS = (512,)
neurons = 512

### Dataset Setup

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH,
    image_size=IMAGE_RESOLUTION,
    seed=SEED,
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH,
    image_size=IMAGE_RESOLUTION,
    seed=SEED,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH,
    image_size=IMAGE_RESOLUTION,
)

class_names = train_ds.class_names

Found 5789 files belonging to 57 classes.
Found 1448 files belonging to 57 classes.
Found 1810 files belonging to 57 classes.


2024-10-01 19:14:31.355466: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-10-01 19:14:31.355497: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-10-01 19:14:31.355507: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-10-01 19:14:31.355521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-01 19:14:31.355531: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Autotune
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Data Augmentation

In [5]:
# Data Augmentation Function
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomContrast(0.2),
        tf.keras.layers.RandomBrightness(0.2),
        tf.keras.layers.RandomCrop(224, 224),
        tf.keras.layers.GaussianNoise(0.1),
    ]
)

### Model Architecutre

In [7]:
base_model = pretrain_models['MOBV2']['model'](
    include_top=False, 
    weights="imagenet",
    pooling='avg', 
    input_shape=IMAGE_RESOLUTION + (3,)
)

base_model.trainable = False

# input
inputs = tf.keras.Input(shape=IMAGE_RESOLUTION + (3,))
x = data_augmentation(inputs) # Data Augmentation Layer
x = pretrain_models['MOBV2']['prep'](x) # PreTrain model image preprocess
x = base_model(x, training=True) # Add base model
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x) # Add drop out layer

#for neurons in NEURONS_LAYERS:
x = tf.keras.layers.Dense(
    neurons, 
    activation='relu',
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.001, l2=0.001)
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
    
output = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=output)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_1 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_1 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 57)             │        29,241 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945,145 (11.23 MB)

 Trainable params: 686,137 (2.62 MB)

 Non-trainable params: 2,259,008 (8.62 MB)

In [14]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint


wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project_v2",
    # track hyperparameters and run metadata with wandb.config
    config={
        "layer_1": neurons,
        "activation_1": "relu",
        "dropout": 0.2,
        "epochs": 5,
        "batch_size": 5,
    },
)

config = wandb.config

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0001
    ),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)



# Train the model
history = model.fit(
    train_ds,
    epochs=config.epochs,
    validation_data=valid_ds,
    callbacks=[
        WandbMetricsLogger(log_freq=5),
        WandbModelCheckpoint(
            "models/my_model.keras"
        ), 
    ],
)

wandb.finish()

batch/accuracy,█▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
batch/loss,▁█▇▆▆▅▅▅▅▅▄▄▃▃▃▂▂▂▁
batch/accuracy,0.82
batch/batch_step,90
batch/loss,5.15709


Epoch 1/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 30s 288ms/step - accuracy: 0.8186 - loss: 4.8625 - val_accuracy: 0.8453 - val_loss: 4.6046
Epoch 2/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 23s 257ms/step - accuracy: 0.8169 - loss: 4.5840 - val_accuracy: 0.8432 - val_loss: 4.3287
Epoch 3/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 24s 259ms/step - accuracy: 0.8271 - loss: 4.2808 - val_accuracy: 0.8488 - val_loss: 4.0802
Epoch 4/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 23s 254ms/step - accuracy: 0.8265 - loss: 4.0625 - val_accuracy: 0.8439 - val_loss: 3.8674
Epoch 5/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 23s 258ms/step - accuracy: 0.8280 - loss: 3.8373 - val_accuracy: 0.8425 - val_loss: 3.6827


In [15]:
wandb.finish()

batch/accuracy,▂▅▅▅▄▄▄▄█▃▄▄▄▅▅▅▁▅▅▅▅▅▅▅▇▅▅▅▅▅▅▅▆▆▅▅▅▄▄▅
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/loss,██████▇▇▆▆▆▆▆▆▆▅▆▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁
epoch/accuracy,▁▇██▇
epoch/epoch,▁▃▅▆█
epoch/loss,█▆▄▃▁
epoch/val_accuracy,▄▂█▃▁
epoch/val_loss,█▆▄▂▁
batch/accuracy,0.82346
batch/batch_step,470
batch/loss,3.79684


In [1]:
x = (224, 224)

x + (3,)

(224, 224, 3)